In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


### Data Scrap

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np



In [5]:
base_url="https://www.emlakjet.com/"

In [6]:
links=[]
for i in range(1,50):
    yeni="https://www.emlakjet.com/satilik-konut/istanbul/"+str(i)+"/"
    html_text=requests.get(yeni).text
    soup = BeautifulSoup(html_text,'lxml')
    models = soup.find_all("div", class_="styles_listingItem__1asTK")
    for model in models:
        link=model.find("a",class_="styles_listingItem__1asTK").get("href")
        links.append(format(link)) 
len(links)

1470

In [7]:
full_link=[]
for l in links:
    full_link.append(base_url+l)
len(full_link)

1470

In [8]:
informations = {"Binanın Yaşı": [], "Binanın Kat Sayısı": [], "Banyo Sayısı": [], "Site İçerisinde": [], "Isıtma Tipi": [], "Kira Getirisi": [], "Balkon Sayısı": [],"Brüt Metrekare":[],"Oda Sayısı":[],"Bulunduğu Kat":[],"Türü":[],"WC Sayısı":[]}

pricess=[]
basliklar=[]
konumlar=[]
semt=[]
mahalle=[]

for x in range(len(full_link)):
    try:
        flag = {"Binanın Yaşı": 0, "Binanın Kat Sayısı": 0, "Banyo Sayısı": 0, "Site İçerisinde": 0, "Isıtma Tipi": 0, "Kira Getirisi": 0, "Balkon Sayısı": 0,"Brüt Metrekare":0,"Oda Sayısı":0,"Bulunduğu Kat":0,"Türü":0,"WC Sayısı":0}
        r = requests.get(full_link[x])

        source = BeautifulSoup(r.content, "lxml")

        columns = source.find_all("div", attrs={"class": "styles_listingInformationTable__1X9Aq"})


        baslik = source.find("h1", attrs={"class": "styles_detailTitle__qBXKm"}).text
        
        tutar = source.find("div", attrs={"class": "styles_price__1e65F"}).text
        tutar = tutar[:tutar.find("TL")]
        
        konum = source.find_all("a", attrs={"class": "styles_breadcrumbItem__3Htz7"})
        
        for i in columns:
            rows = i.find_all("div", attrs={"class": "styles_tableHalfRow__3zO2j"})
            for j in rows:
                rows_i = j.find_all("div", attrs={"class": "styles_tableColumn__2x6nG"})
                description = rows_i[0].text
                if description in list(flag.keys()):
                    informations[description].append(rows_i[1].text)
                    flag[description] = 1
        
        basliklar.append(baslik)    
        pricess.append(tutar)
        semt.append(konum[3].text)
        mahalle.append(konum[4].text)
        keys = list(flag.keys())
        values = list(flag.values())

        for i in range(len(values)):
            if not values[i]:
                informations[keys[i]].append(None)
    except:
        pass


In [48]:
byf=str(informations["Binanın Yaşı"]).replace(" (Yeni)","").replace(" Ve Üzeri","").replace("[","").replace("]","").replace("'","").replace(" ","").split(",")
bks=str(informations["Binanın Kat Sayısı"]).replace("[","").replace("]","").replace("'","").lstrip().replace(" ","").split(",")
bs=str(informations["Banyo Sayısı"]).replace("Yok","0").replace("+","").replace("[","").replace("]","").replace("'","").rstrip().replace(" ","").split(",")
si=str(informations["Site İçerisinde"]).replace("Hayır","0").replace("Evet","1").replace("[","").replace("]","").replace("'","").replace(" ","").lstrip().split(",")
it=str(informations["Isıtma Tipi"]).replace("[","").replace("]","").replace("'","").replace(" ","").split(",")
mk=str(informations["Brüt Metrekare"]).replace("[","").replace("]","").replace("'","").replace(" m2","").replace(" ","").replace(".","").split(",")
kg=str(informations["Kira Getirisi"]).replace("[","").replace("]","").replace("'","").replace(" TL","").replace(" ","").split(",") # 0'lar none'a çevirelecek ortalama alınıp yerleştirilecek YETERSİZ VERİ!
bals=str(informations["Balkon Sayısı"]) # YETERSİZ VERİ
bsl=basliklar
prc=str(pricess).replace("arrow_downward%4","").replace("TL","").replace("'","").replace("[","").replace("]","").replace(",","").split(" ")
smt=str(semt).replace("[","").replace("]","").replace(" Satılık Daire","").replace(" '","").replace("'","").replace(" Satılık Çiftlik Evi","").replace(" Satılık Residence","").replace(" Satılık Villa","").replace(" Satılık Müstakil Ev","").split(",")
mhl=str(mahalle).replace("[","").replace("]","").replace(" Satılık Daire","").replace("'","").replace(" Satılık Çiftlik Evi","").replace(" Satılık Residence","").replace(" Satılık Villa","").replace(" Satılık Müstakil Ev","").split(",")
tur=str(informations["Türü"]).replace("[","").replace("]","").replace(" '","").replace("'","").split(",")
wc=str(informations["WC Sayısı"]).replace("[","").replace("]","").replace(" '","").replace("'","").replace("Yok","0").replace(" None","1").split(",")


### Converting scraped datas into DataFrames 

In [49]:
dftest=pd.DataFrame({
    "İlan Başlığı":bsl,
    "Semt":smt,
    "Mahalle":mhl,
    "Tür":tur,
    "Oda Sayısı":informations["Oda Sayısı"],
    "Binanın Yaşı":byf,
    "Binanın Kat Sayısı":bks,
    "Bulunduğu Kat":informations["Bulunduğu Kat"],
    "Brüt Metrekare":mk,
    "Banyo Sayısı":bs,
    "Wc Sayısı":wc,
    "Site İçerisinde":si,
    "Isıtma Tipi":it,
    "Fiyat":prc,
    
})

In [37]:
dftest.tail(10)

İlan Başlığı  \
1460   Avcılar Cihangir Yeni Bina 2+1 75m2 Bahçe Kat...   
1461         BİGA DAN SANCAKTEPE DE SATILIK 4+1 DUBLEKS   
1462              İnkılap mah İSKANLI ... DUBLEKS daire   
1463  ATAŞEHİR ATATÜRK MAH.İŞ MERK.YOĞUN OLDUĞU BÖLG...   
1464  FLORYA BASINKÖY SATILIK 4+1-240 M2 3 CÜ KAT DE...   
1465  SATILIK DAIRE 2+1 YENI BINA ASANSORLU 430 BIN ...   
1466  ÜMRANİYE BEL.KARŞI SOK LOKASYON DEĞERİ YÜKSEK ...   
1467         SOĞANLIK ORTA MAHALLEDE 1+1 SATILIK DAİRE    
1468                   ASİA'DAN DENİZPARK'TA FIRSAT 4+1   
1469  Atakentte Finans Merkezine yakın satılık ikiz ...   

                       Semt                          Mahalle    Tür  \
1460                Avcılar               Cihangir Mahallesi  Konut   
1461             Sancaktepe                  Fatih Mahallesi  Konut   
1462               Ümraniye                İnkılap Mahallesi  Konut   
1463  Ataşehir Satılık Bina   Atatürk Mahallesi Satılık Bina  Konut   
1464               Bakırköy               Basınköy Mahallesi  Konut   
1465             Eyüpsultan                 Çırçır Mahallesi  Konut   
1466               Ümraniye             Yamanevler Mahallesi  Konut   
1467                 Kartal                   Orta Mahallesi  Konut   
1468                 Pendik               Kaynarca Mahallesi  Konut   
1469               Ümraniye                Atakent Mahallesi  Konut   

     Oda Sayısı Binanın Yaşı Binanın Kat Sayısı Bulunduğu Kat Brüt Metrekare  \
1460        2+1         5-10                  3    Bahçe Katı             75   
1461        4+1            2                  3  Bahçe Dublex            140   
1462        4+2        11-20                  4        4. Kat            160   
1463     8+ Oda        11-20                  5     Düz Giriş            213   
1464        4+1        11-20                  4        3. Kat            250   
1465        2+1            2                  5        2. Kat             85   
1466        2+1            4                  5  Yüksek Giriş             85   
1467        1+1           20                  4     Düz Giriş             65   
1468        4+1        11-20                  8        2. Kat            200   
1469        4+1           20                  3     Düz Giriş            420   

     Banyo Sayısı Wc Sayısı Site İçerisinde    Isıtma Tipi    Fiyat  
1460            1         1               0  KombiDoğalgaz   340000  
1461            1      None               0  KombiDoğalgaz   550000  
1462            3      None               0  KombiDoğalgaz   850000  
1463            0      None               0  KombiDoğalgaz  4200000  
1464            2         2               0  KombiDoğalgaz  9500000  
1465            1         1               0  KombiDoğalgaz   430000  
1466            1      None               0  KombiDoğalgaz   540000  
1467            1         1               0  KombiDoğalgaz   185000  
1468            2      None               0  KombiDoğalgaz  2450000  
1469            2      None               0  KombiDoğalgaz  2800000

In [50]:
dftest["Wc Sayısı"].value_counts()

1    1210
2     219
3      18
0      16
4       6
5       1
Name: Wc Sayısı, dtype: int64

In [12]:
dftest["Semt"].value_counts()

Esenyurt                        186
Beylikdüzü                      131
Bahçelievler                    118
Küçükçekmece                     86
Sancaktepe                       66
Kadıköy                          64
Pendik                           56
Çekmeköy                         53
Ümraniye                         47
Başakşehir                       46
Avcılar                          44
Maltepe                          41
Eyüpsultan                       40
Sultangazi                       40
Kartal                           37
Kağıthane                        33
Fatih                            32
Üsküdar                          32
Gaziosmanpaşa                    31
Şişli                            28
Büyükçekmece                     25
Bakırköy                         24
Sarıyer                          23
Beşiktaş                         22
Sultanbeyli                      21
Güngören                         21
Bağcılar                         18
Bayrampaşa                  

0

In [51]:
dftest.to_excel("dfemlakjet1.xlsx")

In [52]:
dftest.to_csv("dfemlakjet1.csv")